<a href="https://colab.research.google.com/github/jaruizes/AI-CVMatcher/blob/main/ApplicationsScoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importamos las librerías necesarias

In [45]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

Creamos los datos de prueba de la posición de ***"Arquitecto de Soluciones"***


In [160]:
offer_description = "Buscamos un arquitecto de soluciones con experiencia en diseño de arquitecturas escalables en la nube y microservicios."
offer_requirements = [
    {"skill": "Software Architecture", "level": "experto", "desc": "Experiencia en diseño de arquitecturas escalables y resilientes.", "mandatory": "true"},
    {"skill": "Microservices", "level": "experto", "desc": "Implementación de arquitecturas basadas en microservicios.", "mandatory": "true"},
    {"skill": "Cloud", "level": "experto", "desc": "Experiencia con AWS, Azure o GCP", "mandatory": "true"},
    {"skill": "Kubernetes", "level": "medio", "desc": "Despliegue y gestión de aplicaciones en contenedores.", "mandatory": "true"},
    {"skill": "API", "level": "inicial", "desc": "Diseño de APIs REST y GraphQL.", "mandatory": "false"}
]
offer_tasks = ["design scalable and high-performance architectures in the cloud",
               "Asesorar a equipos de desarrollo en mejores prácticas de software",
               "Diseñar e implementar estrategias de integración entre servicios",
               "Garantizar la seguridad y gobernanza de las aplicaciones empresariales"]
offer_tags = ["Architecture", "Cloud", "Microservices", "AWS", "Kubernetes", "Software Design"]

Creamos los datos de prueba de la posición de ***"Desarrollador Front"***



In [141]:
offer_description = "Buscamos un desarrollador frontend con experiencia en React y TypeScript para construir interfaces modernas y escalables."
offer_requirements = [
    {"skill": "React", "level": "experto", "desc": "Experiencia en desarrollo con React y gestión de estado (Redux, Context API).", "mandatory": "true"},
    {"skill": "Typescript", "level": "experto", "desc": "Uso de TypeScript para mejorar la robustez del código.", "mandatory": "true"},
    {"skill": "CSS/SASS", "level": "experto", "desc": "Manejo de estilos avanzados y diseño responsivo.", "mandatory": "true"},
    {"skill": "UI/UX", "level": "inicial", "desc": "Conocimiento en diseño centrado en el usuario.", "mandatory": "false"},
    {"skill": "Testing", "level": "inicial", "desc": "Familiaridad con pruebas unitarias (Jest, React Testing Library).", "mandatory": "false"}
]
offer_tasks = ["Desarrollar interfaces modernas y escalables con React y TypeScript.",
               "Colaborar con diseñadores para mejorar la experiencia de usuario.",
               "Optimizar el rendimiento y accesibilidad de la aplicación.",
               "Implementar pruebas unitarias para asegurar la calidad del código."]
offer_tags = ["Frontend", "React", "TypeScript", "UI/UX", "JavaScript", "Web Development"]

Currículum de **Experto en IA**

In [159]:
test_cv = {
    "summary": "Luis Martínez García is a seasoned professional with over 12 years of experience in developing and implementing artificial intelligence solutions, with a recent focus on Generative AI. He is a recognized speaker at international events on AI, emphasizing how these technologies are transforming industries. Passionate about research and practical applications of advanced AI models, his goal is to lead innovative projects that promote the responsible adoption of artificial intelligence in businesses and organizations. His expertise includes machine learning and deep learning model design, along with proficiency in frameworks such as TensorFlow, PyTorch, and Hugging Face. Additionally, he has experience in cloud data management and MLOps, and is knowledgeable about AI ethics and regulation.",
    "keyPoints": [
        "Over 12 years of experience in AI solutions development.",
        "Expertise in Generative AI and NLP.",
        "Recognized speaker at international AI conferences.",
        "Strong background in machine learning and deep learning.",
        "Experience in MLOps and cloud environments."
    ],
    "weakPoints": [
        "Limited experience outside AI and machine learning.",
        "No formal management experience mentioned.",
        "Potential overemphasis on Generative AI despite broad AI experience.",
        "Focus primarily on technical skills with less emphasis on soft skills.",
        "May require upskilling in emerging AI regulations."
    ],
    "hardSkills": [
        "Machine Learning",
        "Deep Learning",
        "Generative AI",
        "Natural Language Processing (NLP)",
        "MLOps"
    ],
    "softSkills": [
        "Communication",
        "Leadership",
        "Teamwork",
        "Problem-solving",
        "Research"
    ],
    "tasksAndResponsabilities": [
        "Lead development projects based on Generative AI for various sectors.",
        "Create custom models for text and image generation.",
        "Implement AI systems in production platforms using Kubernetes and Docker.",
        "Develop advanced machine learning algorithms for predictive analysis.",
        "Research NLP models and contribute to scientific publications.",
        "Supervise research projects in collaboration with universities.",
        "Design and maintain data pipelines for machine learning projects.",
        "Train models for classification and data analysis."
    ],
    "interviewQuestions": [
        "Can you describe a successful project you led in Generative AI?",
        "How do you approach the ethical considerations in AI?",
        "What challenges have you faced when deploying AI solutions in production?",
        "How do you keep your skills updated in the rapidly evolving AI landscape?",
        "Can you provide an example of how you've collaborated with cross-functional teams?"
    ],
    "totalYearsExperience": 12,
    "tags": [
        "Artificial Intelligence",
        "Generative AI",
        "Machine Learning",
        "Deep Learning",
        "NLP",
        "MLOps"
    ]
}

Currículum de **Arquitecto de Soluciones**

In [145]:
test_cv = {
    "summary": "José Alberto Ruiz Casarrubios is a passionate technology expert with over 20 years of experience in strategic and technical roles across various sectors. He possesses extensive knowledge in Frontend, Backend, Cloud, APIs, and Databases, enabling him to manage complex architectures in high-performance enterprise environments. He emphasizes an end-to-end vision in solution delivery, ensuring that solutions are not only current but also scalable and adaptable for future needs. With experience in mentoring less experienced professionals, he actively contributes to the tech community through articles and presentations at events like Commit Conf and OpenExpo. His role as a Principal Solutions Architect at Paradigma Digital involves participation in pre-sales, designing end-to-end architectures, developing PoCs for new technologies, and standardizing best practices across the organization. He has led significant projects such as the digital transformation of document management for Mercadona and the regulatory adaptation for BME. His technical expertise spans a wide array of technologies, including Angular, Spring, Kubernetes, AWS, and GCP, and he holds multiple relevant certifications. José is committed to continuous learning and sharing knowledge, reflected in his publications and talks on modern architecture.",
    "keyPoints": [
        "Over 20 years of experience in technology roles.",
        "Expertise in end-to-end solution architecture.",
        "Strong mentoring and leadership skills.",
        "Active contributor to the tech community through articles and presentations.",
        "Proficient in a wide range of modern technologies and methodologies."
    ],
    "weakPoints": [
        "Limited recent experience in purely software development roles.",
        "Heavy focus on architecture may reduce hands-on coding experience.",
        "May require adaptation to rapidly changing technology trends.",
        "Potential gaps in experience with very niche technologies.",
        "Less emphasis on purely business-oriented roles."
    ],
    "hardSkills": [
        "Angular",
        "Spring",
        "Kubernetes",
        "Cloud",
        "API",
        "AWS",
        "Software Architecture",
        "Kubernetes",
        "Terraform",
        "Kafka",
        "Java",
        "Microservices",
        "CI/CD"
    ],
    "softSkills": [
        "Communication",
        "Leadership",
        "Mentoring",
        "Problem-solving",
        "Team collaboration"
    ],
    "tasksAndResponsabilities": [
        "Lead architectural design and development of end-to-end solutions.",
        "Conduct pre-sales assessments and client engagements.",
        "Develop and standardize best practices for development across the organization.",
        "Mentor junior staff and support their professional growth.",
        "Participate in technical events and publish related articles.",
        "Manage technical aspects of significant projects from inception to execution.",
        "Design and develop Proofs of Concept (PoCs) for new technologies.",
        "Oversee the technical implementation and provide functional support."
    ],
    "interviewQuestions": [
        "Can you describe your approach to developing end-to-end architectures?",
        "What strategies do you use for mentoring junior team members?",
        "How do you keep up with new technologies and trends in the industry?",
        "Can you discuss a challenging project you managed and how you overcame obstacles?",
        "What role do you believe documentation and standardization play in your projects?"
    ],
    "totalYearsExperience": 20,
    "averagePermanency": 2.5,
    "tags": [
        "Microservices",
        "Architecture",
        "Leadership",
        "Java Expert",
        "Cloud Solutions"
    ]
}

Definimos las funciones de cálculo "TF-IDF cosine similarity" y de "similaridad semántica"

In [137]:
model = SentenceTransformer('all-MiniLM-L6-v2')
model_for_tasks = SentenceTransformer("sentence-transformers/LaBSE")

def compute_tfidf_similarity(job_desc, cv_desc):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([job_desc, cv_desc])
    return cosine_similarity(vectors[0], vectors[1])[0][0]

def compute_semantic_similarity(text1, text2):
    embedding1 = model.encode(text1, convert_to_tensor=True)
    embedding2 = model.encode(text2, convert_to_tensor=True)
    return np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

def compute_semantic_similarity_task(text1, text2):
    embedding1 = model_for_tasks.encode([text1])
    embedding2 = model_for_tasks.encode([text2])
    score = cosine_similarity(embedding1, embedding2)[0][0]
    return score



Definimos la función para realizar el score de los requisitos


In [132]:
def score_requirements(requirements, cv_skills):
    total_score = 0
    max_score = 0

    for req in requirements:
        skill = req["skill"]
        level = req["level"]
        desc = req["desc"]
        mandatory = req["mandatory"]

        best_match = max((compute_semantic_similarity_task(skill, cv_skill) for cv_skill in cv_skills), default=0)
        weight = 2 if mandatory.lower() == 'true' else 1
        total_score += best_match * weight
        max_score += weight

    return total_score / max_score if max_score > 0 else 0

Definimos la función para calcular la similaridad en el apartado de tareas

In [133]:
def score_tasks(tasks, cv_tasks):
    cv_tasks = [task.lower() for task in cv_tasks]
    tasks = [task.lower() for task in tasks]
    if not tasks:
        return 0
    total_score = 0
    for task in tasks:
        best_match = max((compute_semantic_similarity_task(task, cv_task) for cv_task in cv_tasks), default=0)
        #print("Task: " + task + " // Best match: " + str(best_match))

        total_score += best_match

    return total_score / len(tasks)


    #task_similarities = [max((compute_semantic_similarity(task, exp) for exp in cv_tasks), default=0) for task in tasks]
    #return np.mean(task_similarities) if task_similarities else 0
    #return compute_semantic_similarity(str(tasks), str(cv_tasks))


Definimos la función para el cálculo de similaridad de tags

In [134]:
def score_tags(job_tags, cv_skills):
    ##matching_tags = len(set(job_tags) & set(cv_tags))
    ##return matching_tags / len(job_tags) if job_tags else 0
    return compute_semantic_similarity_task(str(job_tags), str(cv_skills))


Definimos la función de **cálculo de scoring**

In [157]:
def compute_cv_score(job_desc, requirements, tasks, tags, cv_desc, cv_keypoints, cv_skills, cv_tasks, cv_tags):
    desc_score = compute_semantic_similarity_task(job_desc, str(cv_keypoints) + cv_desc)
    requirement_score = score_requirements(requirements, cv_skills)
    task_score = score_tasks(tasks, cv_tasks)
    #tag_score = score_tags(tags, cv_skills)

    final_score = (0.2 * desc_score) + (0.6 * requirement_score) + (0.2 * task_score)
    return {
        'desc_score': desc_score,
        'requirement_match': requirement_score,
        'task_match': task_score,
        #'tag_match': tag_score,
        'final_score': final_score
    }

Por último, calculamos el scoring total

In [161]:
cv_skills = test_cv['hardSkills'] + test_cv['softSkills']
score = compute_cv_score(offer_description, offer_requirements, offer_tasks, offer_tags, test_cv['summary'], test_cv['keyPoints'], cv_skills, test_cv['tasksAndResponsabilities'], test_cv['tags'])

print('desc_score: ', score['desc_score'])
print('requirement_match: ', score['requirement_match'])
print('task_match: ', score['task_match'])
#print('tag_match: ', score['tag_match'])
print('final_score: ', score['final_score'])

desc_score:  0.42311776
requirement_match:  0.37393903070025974
task_match:  0.49143779277801514
final_score:  0.4072745283444722
